In [6]:
import investpy as ip
import yahooquery as yq
from pandas_datareader import data as pdr
import pandas as pd
from tqdm import trange
import yfinance as yf


In [2]:
df = pd.read_excel('.\datas\MSCIeurope.xlsx')
df.head()

,Ticker,Ticker Yahoo,Nom,Secteur,Equity class,Market Share,Weights,Valeur notionnelle,Actions,ISIN,Currency
0,NESN,NSRGY,NESTLE SA,Biens de consommation de base,Actions,"EUR 48 274 991,14",4.67,48274991.14,420838,CH0038863350,CHF
1,ASML,ASML,ASML HOLDING NV,Technologie de l'informatioon,Actions,"EUR 34 906 428,20",3.38,34906428.20,60802,NL0010273215,EUR
2,ROG,ROG.SW,ROCHE HOLDING PAR AG,Santé,Actions,"EUR 33 188 774,68",3.21,33188774.68,104989,CH0012032048,CHF
3,MC,MC.PA,LVMH,Biens de consommation cycliques,Actions,"EUR 29 028 191,40",2.81,29028191.40,41457,FR0000121014,EUR
4,NOVOB,NVO,NOVO NORDISK CLASS B,Santé,Actions,"EUR 28 242 417,87",2.73,28242417.87,247375,DK0060534915,DKK


# Get yahoo ticker

In [2]:
def find_ticker(isin):
    try:
        company_name = ip.stocks.search_stocks(by='isin', value=str(isin))
        company_name = company_name["name"][0].split(' ')[0]
        symbol = yq.search(company_name)["quotes"][0]["symbol"]
    except:
        symbol = None
    return symbol

In [ ]:
for i in range(len(df)):
    df.loc[i, "Ticker Yahoo"] = find_ticker(df.loc[i, "ISIN"])
df = df[['Ticker', 'Ticker Yahoo', 'Nom', 'Secteur', 'Equity class', 'Market Share', 'Weights',
       'Valeur notionnelle', 'Actions', 'ISIN', 'Currency', ]] 

In [23]:
df.loc[(df['Ticker Yahoo'].isna()), 'Weights'].sum()

93.16999999999999

In [ ]:
def esg_data(ticker):
    try:
        esg = yf.Ticker(ticker).sustainability
        
    except:
        esg = None
    return esg
esg_data('MC.PA')

In [115]:
# esg scores
def esg_score(ticker):
    try:
        sus = yf.Ticker('ROG.SW').sustainability
        score = pd.DataFrame({'Ticker Yahoo': ticker, 'Environment Score': sus.loc['environmentScore','Value'], 'Social Score': sus.loc['socialScore','Value'], 'Governance Score': sus.loc['governanceScore','Value'], 'Total ESG': sus.loc['totalEsg','Value']})
        #print(esg_scores)
    except:
        score = None
    return score

In [116]:
esg_score('ROG.SW')

In [7]:
esg_scores = pd.DataFrame(columns = ['Ticker Yahoo', 'Environment Score', 'Social Score', 'Governance Score', 'Total Score'], index = range(len(df)))
# trange = tqdm(range(len(df)))

for i in trange(len(df)):
    ticker = df.loc[i, 'Ticker Yahoo']
    try:
        sus = yf.Ticker(ticker).sustainability
        scores = sus.loc[['environmentScore','socialScore','governanceScore','totalEsg'],'Value']
        esg_scores.loc[i] = [ticker, scores[0], scores[1], scores[2], scores[3]]
    except:
        pass
esg_scores



 53%|█████▎    | 189/357 [35:36<37:34, 13.42s/it] 

In [98]:
esg_scores = pd.DataFrame(columns = ['Ticker Yahoo', 'Environment Score', 'Social Score', 'Governance Score', 'Total Score'], index = df['Ticker Yahoo'].dropna())
for i in range(len(df)):
    esg_scores.loc[i] = esg_score(df.loc[i, "Ticker Yahoo"])
esg_scores
# add a row
esg_scores.loc[0] = ['ROG.SW', 1, 2, 3, 4]
scores = sus.loc[['environmentScore','socialScore','governanceScore','totalEsg'],'Value']
#esg_scores['Ticker Yahoo'] = 'ROG.SW'
esg_scores[['Environment Score','Social Score', 'Governance Score', 'Total Score']] = scores

esg_scores
#sus.loc[['environmentScore','socialScore','governanceScore','totalEsg']].values
# add scores to df

,Ticker,Ticker Yahoo,Nom,Secteur,Equity class,Market Share,Weights,Valeur notionnelle,Actions,ISIN,Currency
0,NESN,NSRGY,NESTLE SA,Biens de consommation de base,Actions,"EUR 48 274 991,14",4.67,48274991.14,420838,CH0038863350,CHF
1,ASML,ASML,ASML HOLDING NV,Technologie de l'informatioon,Actions,"EUR 34 906 428,20",3.38,34906428.20,60802,NL0010273215,EUR
2,ROG,ROG.SW,ROCHE HOLDING PAR AG,Santé,Actions,"EUR 33 188 774,68",3.21,33188774.68,104989,CH0012032048,CHF
3,MC,MC.PA,LVMH,Biens de consommation cycliques,Actions,"EUR 29 028 191,40",2.81,29028191.40,41457,FR0000121014,EUR
4,NOVOB,NVO,NOVO NORDISK CLASS B,Santé,Actions,"EUR 28 242 417,87",2.73,28242417.87,247375,DK0060534915,DKK
...,...,...,...,...,...,...,...,...,...,...,...
352,UN01,UN01.DE,UNIPER,Services publics,Actions,"EUR 65 417,65",0.01,65417.65,13623,DE000UNSE018,EUR
353,GBP,NaN,GBP CASH,Liquidités et/ou produits dérivés,Marchés monétaires (tous),"EUR 45 974,45",0.00,45974.45,39560,-,GBP
354,SMZ2,NaN,SWISS MKT IX DEC 22,Liquidités et/ou produits dérivés,Futures,"EUR 0,00",0.00,2041784.88,18,DE000C6SLS41,CHF
355,VGZ2,NaN,EURO STOXX 50 DEC 22,Liquidités et/ou produits dérivés,Futures,"EUR 0,00",0.00,6416820.00,162,DE000C52VJW3,EUR
